In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\opt\\spark\\spark-3.1.1-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
pyspark

<module 'pyspark' from 'C:\\opt\\spark\\spark-3.1.1-bin-hadoop2.7\\python\\pyspark\\__init__.py'>

In [4]:
# Testando o Spark e criando uma RDD

lst = [25, 90, 81, 37, 776, 3320]
testdata = sc.parallelize(lst)

In [5]:
?sc.parallelize

Signature: sc.parallelize(c, numSlices=None)
Docstring:
Distribute a local Python collection to form an RDD. Using range
is recommended if the input represents a range for performance.

Examples
--------
>>> sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()
[[0], [2], [3], [4], [6]]
>>> sc.parallelize(range(0, 6, 2), 5).glom().collect()
[[], [0], [], [2], [4]]
File:      c:\opt\spark\spark-3.1.1-bin-hadoop2.7\python\pyspark\context.py
Type:      method


In [6]:
type(testdata)

pyspark.rdd.RDD

In [7]:
testdata.count()

6

In [8]:
testdata.collect()

[25, 90, 81, 37, 776, 3320]

**RDD's são coleçães distribuídas de itens. RDD's podem ser criadas a partir do Hadoop (Arquivos no HDFS), através da transformação de outras RDD's a partir de bancos de dados(relacionais e não-relacionais) ou a partir de arquivos locais.**

In [9]:
# Criando uma RDD a partir de um arquivo csv

sentimentoRDD = sc.textFile('sentimentos.csv')

In [10]:
type(sentimentoRDD)

pyspark.rdd.RDD

In [11]:
# Ação - contando o número de registros

sentimentoRDD.count()

100

In [12]:
# # listando os 5 primeiros registros

sentimentoRDD.take(5)

['positivo,Esse livro é incrível.',
 'positivo,Um dos melhores livros que eu já li.',
 'positivo,um dos melhores livros que eu já li',
 'positivo,Acho que ele tem um conteúdo que vai além do que está em sua descrição.',
 'positivo,O Sol é para todos é profundo e emocionante']

In [13]:
# Transformando os dados - transformação letras maiúsculas

transfRDD = sentimentoRDD.map(lambda x: x.upper())
transfRDD.take(5)

['POSITIVO,ESSE LIVRO É INCRÍVEL.',
 'POSITIVO,UM DOS MELHORES LIVROS QUE EU JÁ LI.',
 'POSITIVO,UM DOS MELHORES LIVROS QUE EU JÁ LI',
 'POSITIVO,ACHO QUE ELE TEM UM CONTEÚDO QUE VAI ALÉM DO QUE ESTÁ EM SUA DESCRIÇÃO.',
 'POSITIVO,O SOL É PARA TODOS É PROFUNDO E EMOCIONANTE']

In [14]:
sentimentoRDD.take(5)

['positivo,Esse livro é incrível.',
 'positivo,Um dos melhores livros que eu já li.',
 'positivo,um dos melhores livros que eu já li',
 'positivo,Acho que ele tem um conteúdo que vai além do que está em sua descrição.',
 'positivo,O Sol é para todos é profundo e emocionante']

In [15]:
arquivo = sc.textFile('sentimentos.csv')

In [16]:
type(arquivo)

pyspark.rdd.RDD

In [17]:
arquivo.count()

100

In [18]:
arquivo.first()

'positivo,Esse livro é incrível.'

In [19]:
linhasComSol = arquivo.filter(lambda line:'Sol' in line)

In [20]:
type(linhasComSol)

pyspark.rdd.PipelinedRDD

In [21]:
linhasComSol.count()

3

In [22]:
linhasComSol.take(5)

['positivo,O Sol é para todos é profundo e emocionante',
 'negativo,Sol para todos é um Livro ruim',
 'negativo,O Sol para todos não é para todos.']

Primeiro a função **map()** determina o comprimento de cada linha do arquivo, criando uma RDD. A função **reduce()** é chamada para encontar a linha com maior número de caracteres. o argumento para as funções **map() e reduce()** são funções anônimas criadas com lambda(da linguagem Python).

In [23]:
arquivo.map(lambda line: len(line.split())).reduce(lambda a, b: a if (a > b) else b)

27

Esta linha pode ser reescrita da seguinte forma:

In [24]:
def max(a, b):
    if a > b:
        return a
    else:
        return b

arquivo.map(lambda line: len(line.split())).reduce(max)

27

## Operação de MapReduce

As operações de MapReduce foram popularizadas pelo hadoop e podem ser feitas com Spark até 100x mais rápiado.

In [25]:
contaPalavras = arquivo.flatMap(lambda line : linesplit()).map(lambda palavra: (palavra, 1)).reduceByKey(lambda a, b:a+b)

In [26]:
contaPalavras

PythonRDD[20] at RDD at PythonRDD.scala:53

http://localhost:4040/